In [ ]:
from selenium import webdriver               # 웹드라이버 모듈 가져오기
from selenium.webdriver.common.by import By      
from selenium.webdriver.support.ui import WebDriverWait    # 특정 조건이 충족하길 기다리는 모듈
from selenium.webdriver.support import expected_conditions as EC     
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import TimeoutException, NoSuchElementException   
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException



import pandas as pd
import time
import re

In [ ]:
driver = webdriver.Chrome()

# 페이지 열기 - 인프런 온라인 강의 목록
driver.get('https://www.inflearn.com/courses?order=seq&types=ONLINE')

In [ ]:
# 크롤링 

wait = WebDriverWait(driver, 10)
list1 = []
first_iteration = True

while True:
    try:
        for i in range(1, 25):
            title_xp = '//*[@id="courses_section"]/div/div/div/main/div[4]/div/div[{}]/div/a/div[2]/div[1]'.format(i)

            try:
                title = driver.find_element(By.XPATH, title_xp).text
            except NoSuchElementException:
                print(f"강의명을 찾을 수 없습니다. 루프를 종료합니다.")
                break

            # 가격 정보
            price_xp = '//*[@id="courses_section"]/div/div/div/main/div[4]/div/div[{}]/div/a/div[2]/div[4]'.format(i)
            price = driver.find_element(By.XPATH, price_xp).text

            try: # 상세페이지 들어가기
                title_element = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, title_xp))
                )
                driver.execute_script("arguments[0].click();", title_element)

                #난이도
                level = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.XPATH, '//*[@id="description"]/h2[1]/strong[1]'))
                ).text

                # 분류군 꺼내기
                category = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.XPATH, '//*[@id="main"]/section/div[1]/div[1]/div/div/div[2]/div[1]'))
                ).text

                #수강생 수
                student = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.XPATH, '//*[@id="main"]/section/div[1]/div[1]/div/div/div[2]/div[3]/span[2]'))
                ).text

                #강사정보
                instructor = driver.find_element(By.XPATH, '//*[@id="main"]/section/div[1]/div[1]/div/div/div[2]/div[4]').text

                #강의시간
                time = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.XPATH, '//*[@id="main"]/section/div[1]/div[2]/div[2]/div/div[2]'))
                ).text


                # 강의 게시일
                born_update = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.XPATH, '//*[@id="main"]/section/div[3]/div/div/div[1]/div/section[4]/div/span[1]'))
                ).text


                # 마지막업데이트 날짜
                last_update = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.XPATH, '//*[@id="main"]/section/div[3]/div/div/div[1]/div/section[4]/div/span[2]'))
                ).text  


                try:#별점
                    score = WebDriverWait(driver, 10).until(
                        EC.visibility_of_element_located((By.XPATH, '//*[@id="main"]/section/div[1]/div[1]/div/div/div[2]/div[3]/span[1]/strong'))
                    ).text

                except:
                    score = '수강평이 없습니다'

                ## 커리큘럼부분   
                test_list = []
                num = 1
                while True:
                    curri_xpath = f'//*[@id="curriculum"]/div[3]/div/div[{num}]/div[1]/span[3]'
                    try:
                        element = WebDriverWait(driver, 10).until(
                            EC.visibility_of_element_located((By.XPATH, curri_xpath))
                        )
                        test_list.append(element.text)
                        num += 1
                        continue
                    except Exception as e:
                        break
                
                # 커리큘럼 세션에 데이터가 없어 오류를 출력할때 사용
                result_list = [item.split('. ')[1] if len(item.split('. ')) > 1 else '' for item in test_list]
                curriculum = ', '.join(result_list)


                # 페이지 뒤로 가기
                driver.back()

                # list1에 취합
                list1.append([category, title, level, price, student, score, instructor, time, born_update, last_update, curriculum])




            except (NoSuchElementException, TimeoutException) as e:
                print(f"Error for index {i}: {e}")

        # 다음 페이지 버튼 대기
        if first_iteration:
            first_page = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="courses_section"]/div/div/div/footer/nav/div/a')))
            # JavaScript를 사용하여 클릭
            driver.execute_script("arguments[0].click();", first_page)
            first_iteration = False

        else:
            next_page_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="courses_section"]/div/div/div/footer/nav/div/a[2]')))
            driver.execute_script("arguments[0].click();", next_page_button)


    except TimeoutException:
        # TimeoutException이 발생하면 (예: 다음 페이지 버튼이 더 이상 없음)
        print("다음 페이지 버튼이 더 이상 없습니다.")
        break


driver.close()   

In [ ]:
#리스트를 데이터프레임으로 변환
# 컬럼명 선언.
col_name = ['분류', '강좌명', '난이도', '가격', '학생수', '별점', '강사명', '강좌시간', '게시날짜', '마지막업데이트', '커리큘럼']

df = pd.DataFrame(list1)
df.columns = col_name
df.to_csv('메인(전처리x).csv', index=False, encoding='cp949', errors='ignore')